In [1]:
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

In [2]:
df = pd.read_csv('Set3_Dataset_Threshold60.csv')
df.head()

,Keyword,Subreddit,Title,Text,Created_UTC,Score,Num_Comments,URL,Comment_Text,Comment_Created_UTC,...,comment_hashtags_count,comment_mentions_count,numerics_count,emails,emails_count,token_length,Preprocessed_Comment_Text,comment_embedding,relevant_combination,similarity_score
0,health,BabyBumpsCanada,Pocket Health ultrasound images,is anyone else having issues getting their ult...,1.701969e+09,1,7,https://www.reddit.com/r/BabyBumpsCanada/comme...,i had the same issue. their usual time is up t...,1.702002e+09,...,0,0,1,[],0,23,issue usual time 5 business day call pocket he...,[ 4.15552258e-01 -1.93226129e-01 6.91090047e-...,Dependability Medicine/Medical,0.620348
1,health,BabyBumpsCanada,Pocket Health ultrasound images,is anyone else having issues getting their ult...,1.701969e+09,1,7,https://www.reddit.com/r/BabyBumpsCanada/comme...,sometimes they take a long time. i have my mom...,1.701991e+09,...,0,0,0,[],0,28,sometimes take long time mom account wait ultr...,[ 1.23925179e-01 -1.22495957e-01 8.52569640e-...,Mistrust Doctor,0.600642
2,health,BabyBumpsCanada,Pocket Health ultrasound images,is anyone else having issues getting their ult...,1.701969e+09,1,7,https://www.reddit.com/r/BabyBumpsCanada/comme...,seconding this advice - i emailed pockethealth...,1.702005e+09,...,0,0,0,[],0,13,second advice email pockethealth support image...,[ 3.43326271e-01 -5.43862917e-02 6.11860335e-...,Mistrust Medicine/Medical,0.665982
3,health,BabyBumpsCanada,Should I add newborn to health benefits?,i am debating whether or not to add my newborn...,1.692058e+09,11,23,https://www.reddit.com/r/BabyBumpsCanada/comme...,my newborn needed occupational and physiothera...,1.692059e+09,...,0,0,0,[],0,43,newborn need occupational physiotherapy 36 mon...,[-1.46874320e-02 2.38497853e-01 5.00829875e-...,Mistrust Medicine/Medical,0.629016
4,health,BabyBumpsCanada,Should I add newborn to health benefits?,i am debating whether or not to add my newborn...,1.692058e+09,11,23,https://www.reddit.com/r/BabyBumpsCanada/comme...,check with your insurance carrier if they can ...,1.692060e+09,...,0,0,1,[],0,17,check insurance carrier add later mine deadlin...,[ 3.56267542e-01 3.41842286e-02 5.87649405e-...,Underrepresented Clinic,0.654463


In [3]:
# Load the pretrained model, tokenizer, and configuration
MODEL = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Function to predict sentiment and get probabilities
def predict_sentiment(text):
    text = str(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    
    # Move inputs to GPU if available
    encoded_input = {key: value.to(device) for key, value in encoded_input.items()}
    
    # Perform inference
    with torch.no_grad():
        output = model(**encoded_input)
    
    # Extract scores and apply softmax
    scores = output[0][0].detach().cpu().numpy()
    probabilities = softmax(scores)
    
    # Get the sentiment ranking
    ranking = np.argsort(probabilities)[::-1]
    
    # Get the top sentiment
    sentiment = ranking[0]
    
    # Print the labels and scores for debugging
    for i in range(scores.shape[0]):
        l = config.id2label[ranking[i]]
        s = probabilities[ranking[i]]
        print(f"{i+1}) {l} {np.round(float(s), 4)}")
    
    # Return the sentiment and its probabilities
    return sentiment, probabilities

# Convert sentiment integer to label
def sentiment_to_label(sentiment):
    return config.id2label[sentiment]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Run the Following Cell to Calculate Sentiment Probability Scores and Sentiment Labels (Positive, Negative, or Neutral)

In [ ]:
# Apply the sentiment analysis to the DataFrame
sentiment_results = df['Preprocessed_Comment_Text'].apply(predict_sentiment)

# Extract sentiment and probabilities into separate columns
df['Comment_Sentiment'] = sentiment_results.apply(lambda x: x[0])
df['Comment_Sentiment_Label'] = df['Comment_Sentiment'].apply(sentiment_to_label)
df['Comment_Negative_Probability'] = sentiment_results.apply(lambda x: x[1][0])
df['Comment_Neutral_Probability'] = sentiment_results.apply(lambda x: x[1][1])
df['Comment_Positive_Probability'] = sentiment_results.apply(lambda x: x[1][2])